[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Bartcardi/azure_ml_training/blob/duckdb_talk/notebooks/duckdb_talk.ipynb)

In [ ]:
import time
import pandas as pd
import duckdb

# Setup kaggle credentials

 Visit www.kaggle.com. Go to your profile and click on account. On the following page you will see an API section, where you will find a “Create New API Token” click on it, and it will download a kaggle.json file in which you will get your username and key. 

# If on Colab

Follow the instructions found here: [https://www.kaggle.com/discussions/general/74235#2580958](https://www.kaggle.com/discussions/general/74235#2580958) to set Colab secrets found in the downloaded `kaggle.json` file. When the secrets are added and set to notebook access run the cell below.

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

# If *not* on Colab

Copy the downloaded `kaggle.json` to `~/.kaggle`

In [ ]:
!kaggle datasets download -d zanjibar/100-million-data-csv

In [ ]:
!unzip 100-million-data-csv.zip

In [ ]:
!rm 100-million-data-csv.zip custom_1988_2020.db

# "DuckDB: Your New Favorite Analytical Tool"

* Bart Joosten / ilionx
* 12-02-2025

# The Story of DuckDB (Origin)

* DuckDB was born out of academic research at CWI (Centrum Wiskunde & Informatica) in the Netherlands.
* Originally developed by Mark Raasveldt and Hannes Mühleisen.
* The initial goal was to create an in-process database system optimized for analytical queries on embedded devices.
* The project quickly evolved into a powerful and versatile analytical DBMS suitable for a wide range of applications.
* It's open-source (MIT License) and has a vibrant community contributing to its development.


# What is DuckDB?

* DuckDB is an embedded or  analytical database management system (DBMS).
* It's designed to be fast, portable, and easy to use, especially for analytical queries.
* Written in C++ with zero dependies (only working C++11 compiler required).
* Think of it as SQLite for analytics.
* **Key Feature:** Optimized for analytical workloads (OLAP).

# Why Use DuckDB (Especially with Python)?

* **Speed:** DuckDB is *significantly* faster than using Pandas or other Python libraries for many analytical operations, especially on larger datasets.  It pushes computation down into the database engine, which is highly optimized.
* **Ease of Use:**  It's embedded! No setting up a separate database server. Just install the `duckdb` Python package and you're ready to go.
* **SQL Power:** Leverage the full power of SQL for complex queries, aggregations, and data transformations directly within your Python code. This can be more concise and efficient than equivalent Pandas code.
* **Seamless Integration:** The `duckdb` Python library provides a smooth interface for interacting with DuckDB. You can easily load data from Pandas DataFrames, execute SQL queries, and retrieve results back into Pandas.
* **Portability and Reproducibility:**  DuckDB is entirely self-contained.  This makes your analyses portable (serverless) and reproducible.  You can easily share your code and data without worrying about database configurations.
* **Parquet and other formats:** Read and write Parquet, CSV, and other data science-friendly formats directly.


# OLAP vs. OLTP - The Core Difference

* **OLTP (Online Transaction Processing):**
    * Designed for transactional workloads. Think of your online banking system or e-commerce checkout.
    * Focus: High volume of small transactions, data consistency, and speed of individual transactions.
    * Examples: Inserting a new customer, updating an order status.
* **OLAP (Online Analytical Processing):**
    * Designed for complex analytical queries. Think of business intelligence dashboards or data science exploration.
    * Focus: Analyzing large datasets, complex aggregations, and query performance.
    * Examples: Calculating sales trends over time, identifying customer segments.

# OLAP vs. OLTP - Comparison Table

| Feature           | OLTP                               | OLAP                                  |
|-------------------|------------------------------------|---------------------------------------|
| Workload          | Transactions (inserts, updates)      | Analytical queries (SELECTs, aggregations) |
| Data Volume       | Relatively small transactions         | Large datasets                           |
| Query Complexity  | Simple, fast queries                 | Complex queries, aggregations             |
| Performance Goal  | Transaction speed, data consistency  | Query speed, data analysis              |
| Data Changes      | Frequent, small updates              | Infrequent, bulk updates                |
| Data Focus        | Current, operational data            | Historical, analytical data             |


# Why DuckDB is OLAP-focused

* DuckDB's architecture and optimizations are specifically geared towards OLAP workloads.
* **Columnar storage:** Data is stored column-wise, which is much more efficient for analytical queries that often only access a subset of columns. *(Reduced I/O, better compression)*
* **Vectorized query execution:** DuckDB processes data in batches (vectors), leading to significant performance gains.
* **Optimized query planner:** DuckDB's query planner is designed to find the most efficient execution plan for complex analytical queries.
* These features make DuckDB significantly faster than a traditional row-oriented database (like SQLite) for analytical tasks.

# DuckDB Use Cases for Data Scientists

* **Local data analysis:** Analyze large datasets on your laptop without setting up a complex database server.
* **Data exploration and prototyping:** Quickly test out different analytical queries and transformations.
* **Reproducible research:** Embed DuckDB directly into your analysis scripts to ensure reproducibility.
* **Integration with data science tools:** Seamlessly use DuckDB with Python (via the `duckdb` library), R, and other languages.
* **Parquet and CSV support:** Easily import and export data in common data science formats.

# Demo Time!

* Let's see DuckDB in action! 

# Reading the 100 million rows data in pandas

In [ ]:
start = time.time()
df_csv = pd.read_csv('custom_1988_2020.csv')
end = time.time()
print(end - start)

# Reading the same data into duckdb transient database

In [ ]:
con = duckdb.connect()
start = time.time()
con.execute("CREATE TABLE custom_1988_2020 AS SELECT * FROM 'custom_1988_2020.csv'")
end = time.time()
print(end - start)


# Creating a pandas/polars dataframe from table

In [ ]:
start = time.time()
df_pandas = con.execute("SELECT * FROM custom_1988_2020").fetchdf()
end = time.time()
print(end - start)


In [ ]:
start = time.time()
df_polars = con.execute("SELECT * FROM custom_1988_2020").pl()
end = time.time()
print(end - start)

# Running sql directly on pandas dataframe

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': ['x', 'y', 'z']})
con.register("my_pandas_table", df) # Register a Pandas DataFrame as a virtual table
con.execute("SELECT * FROM my_pandas_table").fetchall()

# Storing the table to a persistent db

In [ ]:
con.execute("ATTACH 'persistent.db' AS persistent").fetchall()
con.execute("CREATE TABLE persistent.custom_1988_2020 AS SELECT * FROM custom_1988_2020").fetchall()

In [ ]:
con.close()

# Opening from the persistent storage

In [ ]:
con = duckdb.connect('persistent.db')

In [ ]:
con.execute("DESCRIBE SELECT * FROM custom_1988_2020").df()

# Describing statistics in duckdb and pandas

In [ ]:
start = time.time()
con.execute("SUMMARIZE SELECT * FROM custom_1988_2020").df()
end = time.time()
print(end - start)


In [ ]:
df.describe()

# SQL Power for Data Exploration

Aggregations: Calculate summary statistics (mean, median, standard deviation, etc.) for various columns. Group by different criteria. Example:

In [ ]:
con.execute("SELECT column1, AVG(column2), COUNT(*) FROM custom_1988_2020 GROUP BY column1 ORDER BY COUNT(*) DESC;").df()

Filtering: Select specific subsets of data based on conditions. Example:

In [ ]:
con.execute("SELECT column4, COUNT(column4) FROM custom_1988_2020 GROUP BY column4 ORDER BY COUNT(column4) DESC;").df()

In [ ]:
con.execute("SELECT * FROM custom_1988_2020 WHERE column3 > 150 AND column4 = '870829000';").df()

Window Functions: Demonstrate powerful analytical functions like running totals, moving averages, or ranking. Example:


In [ ]:
con.execute("SELECT column1, column2, ROW_NUMBER() OVER (ORDER BY column2 DESC) as rank FROM custom_1988_2020 LIMIT 10000;").df()

# DuckDB as Unix tool

https://duckdb.org/2024/06/20/cli-data-processing-using-duckdb-as-a-unix-tool.html

## Datasets

We use the four input files capturing information on cities and airports in the Netherlands.

pop.csv, the population of each of the top-10 most populous cities.

area.csv, the area of each of the top-10 most populous cities.

cities-airports.csv, the IATA codes of civilian airports serving given cities.

airport-names.csv, the airport names belonging to given IATA codes.

In [ ]:
! wget https://duckdb.org/data/cli/duckdb-cli-data.zip

In [ ]:
!unzip duckdb-cli-data.zip

## Projecting Columns

Projecting columns is a very common data processing step. Let's take the pop.csv file and project the first and last columns, city and population.

## Unix Shell: cut
In the Unix shell, we use the cut command and specify the file's delimiter (-d) and the columns to be projected (-f).

In [ ]:
!cut -d , -f 1,3 pop.csv

## DuckDB: SELECT

In DuckDB, we can use the CSV reader to load the data, then use the SELECT clause with column indexes (#i) to designate the columns to be projected:

In [ ]:
con.execute("SELECT #1, #3 FROM 'pop.csv';").fetchall()

In [ ]:
con.execute("""
COPY (
    SELECT #1, #3 FROM 'pop.csv'
  ) TO '/dev/stdout/';
""")

## DuckDB: POSITIONAL JOIN

In DuckDB, we can use a POSITIONAL JOIN. This join type is one of DuckDB's SQL extensions and it provides a concise syntax to combine tables row-by-row based on each row's position in the table. Joining the two tables together using POSITIONAL JOIN results in two city columns – we use the EXCLUDE clause to remove the duplicate column:

In [ ]:
con.execute("""
COPY (
    SELECT pop.*, area.* EXCLUDE city
    FROM 'pop.csv'
    POSITIONAL JOIN 'area.csv'
  ) TO '/dev/stdout/';
""")